## Stream features from Web events using Signals

This notebook creates a new feature view using the SDK, tests it on the atomic events table and applies.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

In [ ]:
from snowplow_signals import Signals

sp_signals = Signals()

### Define a new feature

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Feature,
    FilterCombinator,
    FilterCondition,
)

count_page_pings_feature = Feature(
    name="count_page_pings",
    scope="session",
    dtype="INT32",
    events=[
        "iglu:com.snowplowanalytics.snowplow/page_ping/jsonschema/1-0-0"
    ],
    type="counter",
)

visited_pages_feature = Feature(
    name="visited_pages",
    scope="session",
    dtype="STRING_LIST",
    events=[
        "iglu:com.snowplowanalytics.snowplow/page_view/jsonschema/1-0-0"
    ],
    type="unique_list",
    property="page_title",
)

clicked_links_on_homepage_feature = Feature(
    name="clicked_links",
    scope="session",
    dtype="STRING_LIST",
    events=[
        "iglu:com.snowplowanalytics.snowplow/link_click/jsonschema/1-0-1"
    ],
    type="unique_list",
    property="unstruct_event_com_snowplowanalytics_snowplow_link_click_1:targetUrl",
    filter=FilterCombinator(
        combinator="and",
        condition=[
            FilterCondition(
                property="page_urlpath",
                operator="equals",
                value="/",
            ),
        ],
    ),
)

### Wrapping the feature in a feature view

All features need to be included in feature views that can be considered as "tables" of features.

Feature views are immutable and versioned.

In [ ]:
from snowplow_signals import FeatureView, session_entity

feature_view = FeatureView(
    name="my_web_features",
    version=1,
    entities=[
        session_entity,
    ],
    features=[
        count_page_pings_feature,
        visited_pages_feature,
        clicked_links_on_homepage_feature,
    ],
)

### Testing the feature view

Execute the feature view on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
data = sp_signals.test(
    feature_view=feature_view,
    app_ids=["website"],
)
data

### Applying the feature view to Signals

The following block pushes the feature view definition to the Signals API and makes it available for processing.

In [ ]:
from snowplow_signals import Signals

sp_signals = Signals()
sp_signals.apply([feature_view])